# Table of Contents
- [Import Packages](#Packages)
- [Data Cleaning](#Data-Cleaning)


## Packages

In [ ]:
import os
import time
import csv
import subprocess
import psutil
from datetime import datetime
from fuzzywuzzy import fuzz,process



import pandas as pd
import numpy as np
import seaborn as sns
from feature_engine.imputation import CategoricalImputer

from boruta import BorutaPy

from fancyimpute import KNN
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, recall_score,confusion_matrix, ConfusionMatrixDisplay,roc_curve
from sklearn.inspection import PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


import category_encoders as ce
import zipfile
import sweetviz as sv

import xgboost as xgb
import optuna
import optuna.visualization as vis

import matplotlib.pyplot as plt

## Data-Cleaning